# Chains
## 一、LLMChain
最常用的chain

**核心功能**：直接与大型语言模型（如GPT-4、ChatGLM）交互，用于生成或理解自然语言。

**参数**：

- `llm`：语言模型实例（如`OpenAI()`或`ChatZhipuAI()`）。
- `prompt`：提示模板（`PromptTemplate`），定义输入格式和生成规则。
- 
**使用场景**：
- 简单问答、文本生成、代码编写等单步任务。
**代码示例**：

In [5]:
from langchain.chains import LLMChain
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate

llm = OllamaLLM(
  model="llama3.1:8b"
)

prompt_template = "帮我给{product}想三个可以注册的域名"

llm_chain = LLMChain(
  llm=llm,
  prompt=PromptTemplate.from_template(prompt_template)
)

In [6]:
# llm_chain({"product":"AI"})
llm_chain("AI")

{'product': 'AI',
 'text': '很高兴帮助你!\n\n以下是三种可能可用的域名建议：\n\n1.  **yourblog.top**: 这是一个非常简单和易记的域名，可以适用于个人博客或技术博客。\n2.  **aiworld.net**: 这个域名可以让你的网站表现出与人工智能相关的主题，并且容易让用户记住。\n3.  **techzone.pro**: 如果你打算创建一个聚合各种技术信息的网站，这个域名会很适合。\n\n请注意，域名注册费用和可用性可能因提供商而异。建议在选择域名前检查其当前状态是否为可用，并考虑将来可能的需求和品牌身份设想。'}

## 二、**SequentialChain（顺序链）**

**核心功能**：按顺序串联多个步骤，前一步输出作为下一步输入。

**参数**：
- `chains`：子链列表（如`[LLMChain, ToolChain]`）。
- `input_variables`：输入变量名称（需与子链输入匹配）。

**使用场景**：
- 多步骤任务，如先生成问题再验证答案、文本处理流水线。

**示例代码 1  `simpleSequentialChain`**：

In [6]:
# simpleSequentialChain 只支持固定的链路

from langchain.chains import LLMChain
from langchain_ollama.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SimpleSequentialChain

chat_ollama_model = ChatOllama(
  model="llama3.1:8b",
  temperature=0.0
)

# chain  1  起名字
first_prompt = ChatPromptTemplate.from_template(
  "帮我给{product}的公司起一个好听容易记忆的名字？不要建议，只要一个"
)

chain_1 = LLMChain(
  llm=chat_ollama_model,
  prompt=first_prompt,
  verbose=True
)

# chain  2  描述名字
second_prompt = ChatPromptTemplate.from_template(
  "用五个词来描述一下这个公司的名字：{company_name}"
)

chain_2 = LLMChain(
  llm=chat_ollama_model,
  prompt=second_prompt,
  verbose=True
)

 ## 组合 Chain
over_all_chain = SimpleSequentialChain(
  chains=[chain_1, chain_2],
  verbose=True
)

In [7]:
over_all_chain.run("AI教育培训机构")



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 帮我给AI教育培训机构的公司起一个好听容易记忆的名字？不要建议，只要一个

> Finished chain.
"智慧之源"


> Entering new LLMChain chain...
Prompt after formatting:
Human: 用五个词来描述一下这个公司的名字："智慧之源"

> Finished chain.
这里是五个词来描述这个公司的名字：

1. 智慧
2. 源头
3. 知识
4. 创新
5. 前瞻

> Finished chain.


'这里是五个词来描述这个公司的名字：\n\n1. 智慧\n2. 源头\n3. 知识\n4. 创新\n5. 前瞻'

**示例代码 2 `SequentialChain`**：

In [9]:
from langchain.chains import LLMChain
from langchain_ollama.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain

chat_ollama_model = ChatOllama(
  model="llama3.1:8b",
  temperature=0.0
)

# Chain 1 翻译成中文
first_prompt = ChatPromptTemplate.from_template(
  "把下面的内容翻译成英文：\n\n{content}"
)
chain_1st = LLMChain(
  llm = chat_ollama_model,
  prompt = first_prompt,
  verbose=True,
  output_key = "Chinese_Review",
)

# chain 2 对翻译后的英文进行总结摘要
second_prompt = ChatPromptTemplate.from_template(
  "用一句话总结以下内容：\n\n {Chinese_Review}"
)
chain_2nd = LLMChain(
  llm = chat_ollama_model,
  prompt = second_prompt,
  verbose=True,
  output_key = "Chinese_summary",
)

# chain 3 智能识别语言 input_key 是上一个chain的 output_key
third_prompt = ChatPromptTemplate.from_template(
  "下面的内容是哪种语言？ \n\n {Chinese_summary}"
)
chain_3rd = LLMChain(
  llm = chat_ollama_model,
  prompt = third_prompt,
  verbose=True,
  output_key = "Language",
)

# chain 4 针对摘要进行评论
forth_prompt = ChatPromptTemplate.from_template(
  "请使用指定的语言{Language}对以下内容进行回复： \n\n {Chinese_summary}"
)
chain_4th = LLMChain(
  llm = chat_ollama_model,
  prompt = forth_prompt,
  verbose=True,
  output_key = "Reply",
)

# Over_all Chain
#
overall_chain = SequentialChain(
  chains=[chain_1st,chain_2nd,chain_3rd,chain_4th],
  verbose=True,
  input_variables=["content"],
  output_variables=["Chinese_Review","Chinese_summary","Language","Reply"]
)

In [11]:
overall_chain(
  "床前明月光，疑是地上霜。举头望明月，低头思故乡"
)

/var/folders/d6/8jzk9xqn419djvdnx23ft3y40000gn/T/ipykernel_74263/966799136.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 把下面的内容翻译成英文：

床前明月光，疑是地上霜。举头望明月，低头思故乡

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: 用一句话总结以下内容：

 The moon is shining brightly in front of my bed, it looks like frost on the ground. I raise my head to look at the bright moon and lower my head to think about my hometown.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: 下面的内容是哪种语言？ 

 我抬起头望着明亮的月亮，低下头思念故乡。

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: 请使用指定的语言答案：中文对以下内容进行回复： 

 我抬起头望着明亮的月亮，低下头思念故乡。

> Finished chain.

> Finished chain.


{'content': '床前明月光，疑是地上霜。举头望明月，低头思故乡',
 'Chinese_Review': 'The moon is shining brightly in front of my bed, it looks like frost on the ground. I raise my head to look at the bright moon and lower my head to think about my hometown.',
 'Chinese_summary': '我抬起头望着明亮的月亮，低下头思念故乡。',
 'Language': '答案：中文',
 'Reply': '我抬起头望着明亮的月亮，低下头思念故乡。'}